In [3]:
from ipymlwidgets.widgets import Canvas
from ipymlwidgets.utils import demo_image
import numpy as np

image = demo_image()
canvas = Canvas(width=image.shape[1], height=image.shape[0])
canvas.observe(print, "client_size")
canvas.css_width = "10%"
canvas.css_height = "auto"

canvas.set_image(image)
patch = np.full((10, 10, 4), [255, 0, 0, 255], dtype=np.uint8)
canvas.set_patch(25, 5, 10, 10, patch)
display(canvas)

import random
import ipywidgets as W
import time

def update_and_draw_random_patch(b):   

    #with canvas.hold():
    for i in range(2):
        
        # Draw a random colored patch at random position
        patch_size = random.randint(5, 15)
        x = random.randint(0, canvas.width - patch_size)
        y = random.randint(0, canvas.height - patch_size)
        
        # Random color
        color = [random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 255]
        patch = np.full((patch_size, patch_size, 4), color, dtype=np.uint8)
        
        canvas.set_patch(x, y, patch_size, patch_size, patch)
        print(f"Drew {patch_size}x{patch_size} patch at ({x}, {y}) with color {color[:3]}")
            
# Create the button
update_button = W.Button(description="Update & Draw Random Patch")
update_button.on_click(update_and_draw_random_patch)

display(update_button)

Canvas(client_size=(0, 0), css_width='10%', height=32, width=32)

Button(description='Update & Draw Random Patch', style=ButtonStyle())

{'name': 'client_size', 'old': (0, 0), 'new': (91, 91), 'owner': Canvas(client_size=(91, 91), css_width='10%', height=32, width=32), 'type': 'change'}
Drew 14x14 patch at (13, 1) with color [103, 248, 45]
Drew 7x7 patch at (10, 15) with color [95, 30, 172]


In [ ]:
from ipymlwidgets.widgets import Image
from ipymlwidgets.utils import demo_image


image = Image(demo_image())
image[10:20, 10:20] = [255, 0, 0, 255]
image.observe(print, "client_size")
image.css_width = "50%"
image.css_height = "auto"

display(image)


